In [19]:
# Data COllection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

# Loading the dataset

data = gutenberg.raw('shakespeare-hamlet.txt')

# Save in the file

with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [20]:
# Data Preprocessing
# import tensorflow as tf
# import keras
# import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load the dataset

with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

# Tokenize the datset
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [21]:
# create input sequences

input_sequences = []

for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]

  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]

    input_sequences.append(n_gram_sequence)

In [22]:
input_sequences[:5]

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886]]

In [23]:
# Pad Sequence
import numpy as np
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len


14

In [24]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [25]:
import tensorflow as tf

X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [26]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [27]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [28]:
y =tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
# Splitting the dataset in train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
print(total_words)
print(max_sequence_len)

4818
14


In [32]:
# Training our LSTM model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compiling the model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, max_sequence_len))
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 14, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 100)                 │         100,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Train the model

history = model.fit(X_train, y_train, epochs=50,validation_data = (X_test, y_test),verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 46s 64ms/step - accuracy: 0.0303 - loss: 7.1563 - val_accuracy: 0.0336 - val_loss: 6.6984
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 38s 59ms/step - accuracy: 0.0384 - loss: 6.4750 - val_accuracy: 0.0410 - val_loss: 6.7622
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.0428 - loss: 6.3602 - val_accuracy: 0.0470 - val_loss: 6.7892
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - accuracy: 0.0531 - loss: 6.1624 - val_accuracy: 0.0538 - val_loss: 6.8077
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 60ms/step - accuracy: 0.0534 - loss: 5.9975 - val_accuracy: 0.0579 - val_loss: 6.8707
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.0622 - loss: 5.8638 - val_accuracy: 0.0593 - val_loss: 6.9061
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 42s 62ms/step - accuracy: 0.0703 - loss: 5.7294 - val_accuracy: 0.0637 - val_loss: 6.9759
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - accuracy: 0.0792 - loss: 5.6017 - 

In [37]:
# Prediction

def predict_next_word(model, tokenizer, text, max_sequence_len):
  token_list = tokenizer.texts_to_sequences([text])[0]

  if len(token_list) >= max_sequence_len:
    token_list= token_list[-(max_sequence_len-1):]

  token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
  predicted = model.predict(token_list, verbose = 0)
  predicted_word_index = np.argmax(predicted, axis =1)

  for word, index in tokenizer.word_index.items():
    if index == predicted_word_index:
      return word

  return None

In [38]:
input_text = 'To be or not to be'
print(f'Input Text : {input_text}')
max_sequence_len = model.input_shape[1] +1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f'Next Word : {next_word}')

Input Text : To be or not to be
Next Word : buried


In [41]:
# Saving the model
model.save("next_word_LSTM.h5")

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
!pip install streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

In [48]:
%%writefile app.py
import streamlit as st

st.title("Streamlit on Colab with Cloudflare")
st.write("If you're seeing this, the app is working!")

Overwriting app.py


In [49]:
!streamlit run app.py &> logs.txt &
!./cloudflared tunnel --url http://localhost:8501 --log-level debug

/bin/bash: line 1: ./cloudflared: No such file or directory
